## Gap Data, Part 2 

#### Data Documentation:
<br>**Description**: Synthetic dataset from Gap Inc., representing a random sample of individual purchases from Q1 FY2020. <br>Each row is a unique item purchased in an order
<br><br>

| **Feature** | **Description**    | **Sample Value(s)**  |
| ------- | -----------    | ------------- |
| OrderID | Unique identifier per transaction (7-digit) | DRW7C20   |
| CustomerID | Unique identifier per customer (5-digit) | KP441   |
| ProductID  | Unique identifier per item (8-digit) | 13-817-239 |
| StoreID | Unique identifier per store (4-digit) | #4176 |
| OrderType | How purchase was completed  | InStore, HomeDelivery, Online |
| Timestamp | Timestamp of transaction (YYYY-MM-DD) | 2020-01-18 10:13:56	 |
| Brand | Which reporting segment of Gap Inc. bought from | Banana Republic |
| ItemSize | Size of item | XS, S, M, L, X, XL |
| ProductName | Name of item associated with item identifier | Pink Polo by Kanye |
| Collection | Which part of store | Denim Shop |
| Price | Listed price of item | $29.95 |
| ClearanceType | Type of clearance | Retail, Clearance, Final Sale |
| DiscountType | If Gap Card rewards was used | Reward points, Promotion, GapCash, Other |
| StoreName | Store name (i.e. Mall), or facility where online order was shipped from | Barracks Shopping Center |
| Address | Store location | Charlottesville, VA |
| Lat | Store latitude | 38.054034 |
| Lon | Store longitude | -78.499492 |

<br>


One common task that data folks have to do is **merge data from various sources**.

<bR>For example, it'd be a pretty big waste of space on a database to keep listing all the attributes of a store for each item, if we could store them seperately and link them back up when we need to.

<br>This kind of file is sometimes known as a **crosswalk** or **lookup table**. We have individual observations (Gap sales data), and we want to link it up to information about the store.

### Data & Package Imports

In [9]:
import pandas as pd
import numpy as np

In [201]:
base_url = 'https://raw.githubusercontent.com/ishaandey/node/master/week-3/lab/'
sales = pd.read_csv(base_url+'gap_sales.csv', sep='|')
store_lookup = pd.read_csv(base_url+'gap_stores.csv', sep='|')
locations = pd.read_csv(base_url+'gap_store_locations.csv', sep='|')

## Store Data

### Fill Missing Values

If you look carefully at the stores, you'll notice that for both the Gap and Banana Republic stores, the StoreName value is empty.

<br>Turns out, there was a mistake made when correcting a typo in the database, and now all stores for 'Williamsburg Premium Outlets' are showing missing. 

1. Check the entries for StoreID `#4479` and `#1812` in the store lookup table
2. Fill in the missing value for **store name** (not whole dataframe)
3. Double check that stores #1812 and #4479 no longer have a missing value for either field

### Merge to Location Data

1. Take a look at both the store lookup and location tables. There's only 15 locations, but 25 stores- That's because some locations have both Banana and Gap stores at the same site.

2. Take a look at both dataframes. What columns do they have in common? <br>(Hint: The column name may not be the same, but the values are)

We're now ready to complete the merge, to have one consolidated table that has both the store IDs, and store locations for each store in Virginia.

3. Use the `pd.merge()` function to merge both dataframes together, and specify the keys using the columns you found above (pass these to the `left_on` and `right_on` parameters)

4. Drop the `Store` column from this merged dataframe, since it's now redundant.

## Merging it all

Now that we have the store data cleaned up, let's merge it back onto the main dataset, `sales`.

1. Check which columns are in common between both datasets. (Hint: Think ID, not name)

2. Drop one of the duplicated `Brand` columns in `stores`

3. Merge the `sales` and `stores` dataframe into `df`

## Date Times

Our goal: Isolate sales that were made at various timepoints. The issue is that our transaction timestamp isn't very helpful as a string. 

1. Process the `Timestamp` column, so that it's now a datetime object.

2. How many items were sold in the month of February?

3. What were the total sales after 4:00 PM on Valentine's Day? <br>(Hint: 4pm is 16 on a 24 hour scale)

4. What was the most popular item sold on Valentine's Day?

## Cleaning Data

Sometimes, we'll have to manually adjust data. At gap, the prices indicate the type of sale the item is on. For example, prices that end in `.97` are on final sale, and can't be returned.

1. Using the set of rules below, create a dictionary that maps price endings to the clearance type. Remember, dictionary keys must be strings, so `99` should really be `'99'` 

99 => Clearance; 97 => Final Sale; 95 => Retail Price

2. In order to recognize the price endings, we've gotta (first) convert the floats into strings, (then) extract the numbers following the decimal, (and lastly) save the information to a new column, `ClearanceType`.

3. Finally, we'll map the price endings to type of clearance, using the dictionary we made earlier. Save this to `ClearanceType`.

4. Check the value counts of `ClearanceType` – which type is the most common?


## Working With Numerics

As it turns out, prices don't exactly reflect discounts. Let's work through and create more accurate numbers for revenue.

1. Create a `Discounts` column. If an item is on clearance, we'll apply a rate of 10%, else 0. Use a `map()` or `apply()` function to accomplish this.


2. Calculate a new column, `Paid`, that takes into account a flat tax rate of 6% and the discounts from our previous column.

3. What is the mean absolute difference between the amount paid and the listed price?